In [ ]:
import pandas as pd
import math
import numpy as np
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import re

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# 실적데이터에서 필요한 feature들을 추가한 base_line 엑셀 파일을 불러온다.
df = pd.read_excel('./bc/base_line.xlsx', sheet_name = 'changed_data', header=0)

In [ ]:
df

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,sales,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,YEAR_DAY,DAY_NAME,DAY_NUM,code,p_group_code,day_of_week_and_hour,no,rain,52_week,holiday,brand_name,brand_power,mask_ratio,parcel_ratio,quarter,timeslot,season
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000,53,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0,30,0,0,1,1,테이트,16.386845,0.04095,0.52582,1,1,4
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000,110,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0,30,1,0,1,1,테이트,16.386845,0.04095,0.52582,1,1,4
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000,82,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,0,30,2,0,1,1,테이트,16.386845,0.04095,0.52582,1,1,4
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000,174,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,0,30,3,0,1,1,테이트,16.386845,0.04095,0.52582,1,1,4
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000,167,2019,1,1,6,40,0,2019-01-01,Tuesday,2,0,0,30,4,0,1,1,테이트,16.386845,0.04095,0.52582,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000,69,2019,12,31,23,40,0,2019-12-31,Tuesday,2,22,2,47,37367,0,53,0,쿠첸,13.968137,0.03906,1.28643,4,6,4
35375,2019-12-31 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000,87,2019,12,31,0,0,0,2019-12-31,Tuesday,2,22,2,48,37368,0,53,0,쿠첸,13.968137,0.03949,0.56837,1,7,4
35376,2019-12-31 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000,286,2019,12,31,0,0,0,2019-12-31,Tuesday,2,22,2,48,37369,0,53,0,쿠첸,13.968137,0.03949,0.56837,1,7,4
35377,2019-12-31 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000,621,2019,12,31,0,0,0,2019-12-31,Tuesday,2,22,2,48,37370,0,53,0,쿠첸,13.968137,0.03949,0.56837,1,7,4


In [ ]:
df['fill_exp_min'] = df['exp_min'].fillna(method='ffill')

option_count= []
exp = df['exp_min']

for idx, i in enumerate(exp) :
    
    if math.isnan(float(i)) is False :
        option_count.append(1)
        count = 1
    else:
        count = count + 1
        option_count.append(count)

df['option_count'] = option_count

div_exp_min = []
exp = df['exp_min']
fill_ex = df['fill_exp_min']
count = df['option_count']

for idx, i in enumerate(exp) :
    if idx != len(exp)-1 : 
    
        if count[idx] == 1 and count[idx+1] != 1:
            div_exp_min.append(float('nan'))
        elif count[idx] == 1 and count[idx+1] == 1:
            div_exp_min.append(exp[idx])
        elif count[idx] !=1 and count[idx+1] !=1 :
            div_exp_min.append(float('nan'))
        elif count[idx] !=1 and count[idx+1] ==1:
            div_exp_min.append(fill_ex[idx] / count[idx])
    else:
        div_exp_min.append(fill_ex[idx] / count[idx])

df['div_exp_min'] = div_exp_min
df['div_exp_min'] = df['div_exp_min'].fillna(method='bfill')
del df['option_count']

In [ ]:
# 모델을 돌릴때 불필요한 feature 들을 제거해준다.
df.drop(['broadcast', 'exp_min', 'p_name', 'p_group', 'sales', 'YEAR', 'SECOND', 'YEAR_DAY', 'DAY_NAME', 'code', 'no', 'quarter'], axis='columns', inplace=True)

In [ ]:
df = df[['m_code', 'p_code', 'unit_price', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'DAY_NUM', 'p_group_code', 'day_of_week_and_hour', 'rain', '52_week', 'holiday', 'brand_name', 'brand_power', 'mask_ratio', 'parcel_ratio', 'timeslot', 'season', 'fill_exp_min', 'div_exp_min', 'total_price']]

In [ ]:
# 평가데이터에서 필요한 feature들을 추가한 base_line_eval 엑셀 파일을 불러온다.
df_e = pd.read_excel('./bc/base_line_eval.xlsx', sheet_name = 'changed_data', header=0)

In [ ]:
df_e

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,YEAR_DAY,DAY_NAME,DAY_NUM,code,p_group_code,day_of_week_and_hour,no,52_week\t,holiday,quarter,timeslot,season,rain,mask_ratio,parcel_ratio
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,2020,6,1,6,20,0,2020-06-01,Monday,1,0,0,6,0,23,0,2,1,2,0,1.85848,1.15168
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,2020,6,1,6,40,0,2020-06-01,Monday,1,0,0,6,1,23,0,2,1,2,0,1.85848,1.15168
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,2020,6,1,7,0,0,2020-06-01,Monday,1,0,0,7,2,23,0,2,1,2,0,1.85848,1.15168
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,2020,6,1,7,20,0,2020-06-01,Monday,1,1,1,7,3,23,0,2,1,2,0,1.85848,1.15168
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,2020,6,1,7,40,0,2020-06-01,Monday,1,1,1,7,4,23,0,2,1,2,0,1.85848,1.15168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,2020,7,1,0,20,0,2020-07-01,Wednesday,3,174,5,48,2886,27,0,2,7,2,0,1.11449,1.81406
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,2020,7,1,0,40,0,2020-07-01,Wednesday,3,174,5,48,2887,27,0,2,7,2,0,1.11449,1.81406
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,2020,7,1,1,0,0,2020-07-01,Wednesday,3,174,5,49,2888,27,0,2,7,2,0,1.11449,1.81406
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,2020,7,1,1,20,0,2020-07-01,Wednesday,3,224,0,49,2889,27,0,2,7,2,0,1.11449,1.81406


In [ ]:
df_e['fill_exp_min'] = df_e['exp_min'].fillna(method='ffill')

option_count= []
exp = df_e['exp_min']

for idx, i in enumerate(exp) :
    
    if math.isnan(float(i)) is False :
        option_count.append(1)
        count = 1
    else:
        count = count + 1
        option_count.append(count)

df_e['option_count'] = option_count

div_exp_min = []
exp = df_e['exp_min']
fill_ex = df_e['fill_exp_min']
count = df_e['option_count']

for idx, i in enumerate(exp) :
    if idx != len(exp)-1 : 
    
        if count[idx] == 1 and count[idx+1] != 1:
            div_exp_min.append(float('nan'))
        elif count[idx] == 1 and count[idx+1] == 1:
            div_exp_min.append(exp[idx])
        elif count[idx] !=1 and count[idx+1] !=1 :
            div_exp_min.append(float('nan'))
        elif count[idx] !=1 and count[idx+1] ==1:
            div_exp_min.append(fill_ex[idx] / count[idx])
    else:
        div_exp_min.append(fill_ex[idx] / count[idx])

df_e['div_exp_min'] = div_exp_min
df_e['div_exp_min'] = df_e['div_exp_min'].fillna(method='bfill')
del df_e['option_count']

In [ ]:
!git clone https://github.com/pingpong-ai/chatspace.git
!pip install ./chatspace

Cloning into 'chatspace'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 207 (delta 11), reused 8 (delta 3), pack-reused 173
Receiving objects: 100% (207/207), 9.15 MiB | 36.31 MiB/s, done.
Resolving deltas: 100% (105/105), done.
Processing ./chatspace
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 317kB 2.8MB/s 
  Created wheel for chatspace: filename=chatspace-1.0.2-cp36-none-any.whl size=10669953 sha256=0984ecf8ed1d51c5a7acaea4baa201d77919048ad6afab4b4a4227fb49b9887d
  Stored in directory: /tmp/pip-ephem-wheel-cache-5mqrqa0w/wheels/ea/b5/37/d03de9763c6d2e1d71c30fee9f14287ea59a747abde5902043
Successfully built chatspace
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81699 sha256=8589b4229089949fe116b3d9ef8cd69b0e1d78

In [ ]:
from chatspace import ChatSpace

spacer = ChatSpace()

In [ ]:
df2 = pd.read_excel('./bc/base_line.xlsx', header=0, sheet_name='original_data')

In [ ]:
temp = df2['p_name']

from tqdm import tqdm_notebook

cleaned_name_list = []
for name in tqdm_notebook(temp) :
    
        cleaned_name = re.sub('[\(\)\[\]］,\"\/\+&]', ' ', name)

        cleaned_name = re.sub('  ', ' ' , cleaned_name)
        cleaned_name = cleaned_name.strip()
        cleaned_name = cleaned_name.lower()
        cleaned_name = cleaned_name.replace('김치냉장고', '김치 냉장고')
        cleaned_name = cleaned_name.replace('uhdtv', 'uhd tv')
        cleaned_name = re.split('\s+', cleaned_name)

        new_cleand_name = []
        for words in cleaned_name :
            words = words.lower()
            
            if words != '*' and words != 'x':
                new_cleand_name.append(words)
                
        joined = " ".join(new_cleand_name)
        spaced = spacer.space(joined)
  
        cleaned_name_list.append(spaced)

df2['split_p_name'] = cleaned_name_list
df2['split_p_name'] = [ re.sub('[\[\]]', '', str(name)) for name in cleaned_name_list ]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
test = pd.read_excel('./bc/base_line_eval.xlsx')

In [ ]:
temp2 = test['p_name']

cleaned_name_list = []
for name in tqdm_notebook(temp2) :
    
        cleaned_name = re.sub('[\(\)\[\]］,\"\/\+&]', ' ', name)

        cleaned_name = re.sub('  ', ' ' , cleaned_name)
        cleaned_name = cleaned_name.strip()
        cleaned_name = cleaned_name.lower()
        cleaned_name = cleaned_name.replace('김치냉장고', '김치 냉장고')
        cleaned_name = cleaned_name.replace('uhdtv', 'uhd tv')
        cleaned_name = re.split('\s+', cleaned_name)
        
        new_cleand_name = []
        for words in cleaned_name :
            words = words.lower()
            
            if words != '*' and words != 'x':
                new_cleand_name.append(words)
                
        joined = " ".join(new_cleand_name)
        spaced = spacer.space(joined)

        cleaned_name_list.append(spaced)

test['split_p_name'] = [ re.sub('[\[\]］]', '', str(name)) for name in cleaned_name_list ]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
df2['brand_name_code']=pd.Categorical(df2['brand_name']).codes
df2['u'] = [str(round(i, -4)) for i in df2['unit_price']]
df2['group_p_name'] = df2['p_group'] + ' ' + df2['split_p_name'] +' ' + df2['u']

test['u'] = [str(round(i, -4)) for i in test['unit_price']]
test['group_p_name'] = test['p_group'] + ' ' + test['split_p_name'] + ' ' + test['u']

건강기능 =df2[df2['p_group'] == '건강기능']
생활용품 =df2[df2['p_group'] == '생활용품']
침구 =df2[df2['p_group'] == '침구']
주방 =df2[df2['p_group'] == '주방']
가전 =df2[df2['p_group'] == '가전']
가구 =df2[df2['p_group'] == '가구']
이미용 =df2[df2['p_group'] == '이미용']
농수축 =df2[df2['p_group'] == '농수축']
잡화 =df2[df2['p_group'] == '잡화']
속옷= df2[df2['p_group'] == '속옷']
의류=df2[df2['p_group'] == '의류']

건강기능_test = test[test['p_group'] == '건강기능']
생활용품_test = test[test['p_group'] == '생활용품']
침구_test = test[test['p_group'] == '침구']
주방_test = test[test['p_group'] == '주방']
가전_test = test[test['p_group'] == '가전']
가구_test = test[test['p_group'] == '가구']
이미용_test = test[test['p_group'] == '이미용']
농수축_test = test[test['p_group'] == '농수축']
잡화_test = test[test['p_group'] == '잡화']
속옷_test = test[test['p_group'] == '속옷']
의류_test = test[test['p_group'] == '의류']
무형_test = test[test['p_group'] == '무형']

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook

p_group_name = ['의류', '속옷', '농수축', '이미용', '가전', '가구', '침구', '생활용품', '잡화', '건강기능', '주방']

names = {}
scores = {}

for group in tqdm_notebook(p_group_name) :
    
    if group == '건강기능':
        p_name = np.unique(건강기능['group_p_name'])

        vectorizer = CountVectorizer(ngram_range=(1,5))
        feature_vector = vectorizer.fit_transform(p_name)
        print(feature_vector.shape)
        vocab = vectorizer.get_feature_names()

        transformer = TfidfTransformer(smooth_idf=False)
        feature_tfidf = transformer.fit_transform(feature_vector)

        test_name = 건강기능_test['group_p_name']
        df_len = len(p_name)

        result_score = []
        result_name = []
        
        for name in test_name:
            query = [name]
            q = vectorizer.transform(query)

            sim = []
            for i in range(df_len):
                c = cosine_similarity(q, feature_tfidf[i])
                sim.append(c)

            sim_idx = np.argmax(sim)   
            sim_score = np.max(sim)

            result_score.append(sim_score)
            result_name.append(list(p_name)[sim_idx])
        names['건강기능'] = result_name
        scores['건강기능'] = result_score
        
    elif group == '의류':
        p_name = np.unique(의류['group_p_name'])

        vectorizer = CountVectorizer(ngram_range=(1,5))
        feature_vector = vectorizer.fit_transform(p_name)
        print(feature_vector.shape)
        vocab = vectorizer.get_feature_names()

        transformer = TfidfTransformer(smooth_idf=False)
        feature_tfidf = transformer.fit_transform(feature_vector)

        test_name = 의류_test['group_p_name']
        df_len = len(p_name)

        result_score = []
        result_name = []
        
        for name in test_name:
            query = [name]
            q = vectorizer.transform(query)

            sim = []
            for i in range(df_len):
                c = cosine_similarity(q, feature_tfidf[i])
                sim.append(c)

            sim_idx = np.argmax(sim)   
            sim_score = np.max(sim)

            result_score.append(sim_score)
            result_name.append(list(p_name)[sim_idx])
        names['의류'] = result_name
        scores['의류'] = result_score
        
    elif group == '속옷':
        p_name = np.unique(속옷['group_p_name'])

        vectorizer = CountVectorizer(ngram_range=(1,5))
        feature_vector = vectorizer.fit_transform(p_name)
        print(feature_vector.shape)
        vocab = vectorizer.get_feature_names()

        transformer = TfidfTransformer(smooth_idf=False)
        feature_tfidf = transformer.fit_transform(feature_vector)

        test_name = 속옷_test['group_p_name']
        df_len = len(p_name)

        result_score = []
        result_name = []
        
        for name in test_name:
            query = [name]
            q = vectorizer.transform(query)

            sim = []
            for i in range(df_len):
                c = cosine_similarity(q, feature_tfidf[i])
                sim.append(c)

            sim_idx = np.argmax(sim)   
            sim_score = np.max(sim)

            result_score.append(sim_score)
            result_name.append(list(p_name)[sim_idx])
        names['속옷'] = result_name
        scores['속옷'] = result_score
        
    elif group == '농수축':
        p_name =np.unique(농수축['group_p_name'])

        vectorizer = CountVectorizer(ngram_range=(1,5))
        feature_vector = vectorizer.fit_transform(p_name)
        print(feature_vector.shape)
        vocab = vectorizer.get_feature_names()

        transformer = TfidfTransformer(smooth_idf=False)
        feature_tfidf = transformer.fit_transform(feature_vector)

        test_name = 농수축_test['group_p_name']
        df_len = len(p_name)

        result_score = []
        result_name = []
        
        for name in test_name:
            query = [name]
            q = vectorizer.transform(query)

            sim = []
            for i in range(df_len):
                c = cosine_similarity(q, feature_tfidf[i])
                sim.append(c)

            sim_idx = np.argmax(sim)   
            sim_score = np.max(sim)

            result_score.append(sim_score)
            result_name.append(list(p_name)[sim_idx])
        names['농수축'] = result_name
        scores['농수축'] = result_score
        
    elif group == '이미용':
        p_name = np.unique(이미용['group_p_name'])

        vectorizer = CountVectorizer(ngram_range=(1,5))
        feature_vector = vectorizer.fit_transform(p_name)
        print(feature_vector.shape)
        vocab = vectorizer.get_feature_names()

        transformer = TfidfTransformer(smooth_idf=False)
        feature_tfidf = transformer.fit_transform(feature_vector)

        test_name = 이미용_test['group_p_name']
        df_len = len(p_name)

        result_score = []
        result_name = []
        
        for name in test_name:
            query = [name]
            q = vectorizer.transform(query)

            sim = []
            for i in range(df_len):
                c = cosine_similarity(q, feature_tfidf[i])
                sim.append(c)

            sim_idx = np.argmax(sim)   
            sim_score = np.max(sim)

            result_score.append(sim_score)
            result_name.append(list(p_name)[sim_idx])
        names['이미용'] = result_name
        scores['이미용'] = result_score
        
    elif group == '생활용품':
        p_name = np.unique(생활용품['group_p_name'])

        vectorizer = CountVectorizer(ngram_range=(1,5))
        feature_vector = vectorizer.fit_transform(p_name)
        print(feature_vector.shape)
        vocab = vectorizer.get_feature_names()

        transformer = TfidfTransformer(smooth_idf=False)
        feature_tfidf = transformer.fit_transform(feature_vector)

        test_name = 생활용품_test['group_p_name']
        df_len = len(p_name)

        result_score = []
        result_name = []
        
        for name in test_name:
            query = [name]
            q = vectorizer.transform(query)

            sim = []
            for i in range(df_len):
                c = cosine_similarity(q, feature_tfidf[i])
                sim.append(c)

            sim_idx = np.argmax(sim)   
            sim_score = np.max(sim)

            result_score.append(sim_score)
            result_name.append(list(p_name)[sim_idx])
        names['생활용품'] = result_name
        scores['생활용품'] = result_score    

    elif group == '가구':
        p_name = np.unique(가구['group_p_name'])

        vectorizer = CountVectorizer(ngram_range=(1,5))
        feature_vector = vectorizer.fit_transform(p_name)
        print(feature_vector.shape)
        vocab = vectorizer.get_feature_names()

        transformer = TfidfTransformer(smooth_idf=False)
        feature_tfidf = transformer.fit_transform(feature_vector)

        test_name = 가구_test['group_p_name']
        df_len = len(p_name)

        result_score = []
        result_name = []
        
        for name in test_name:
            query = [name]
            q = vectorizer.transform(query)

            sim = []
            for i in range(df_len):
                c = cosine_similarity(q, feature_tfidf[i])
                sim.append(c)

            sim_idx = np.argmax(sim)   
            sim_score = np.max(sim)

            result_score.append(sim_score)
            result_name.append(list(p_name)[sim_idx])
        names['가구'] = result_name
        scores['가구'] = result_score   

    elif group == '주방':
        p_name = np.unique(주방['group_p_name'])

        vectorizer = CountVectorizer(ngram_range=(1,5))
        feature_vector = vectorizer.fit_transform(p_name)
        print(feature_vector.shape)
        vocab = vectorizer.get_feature_names()

        transformer = TfidfTransformer(smooth_idf=False)
        feature_tfidf = transformer.fit_transform(feature_vector)

        test_name = 주방_test['group_p_name']
        df_len = len(p_name)

        result_score = []
        result_name = []
        
        for name in test_name:
            query = [name]
            q = vectorizer.transform(query)

            sim = []
            for i in range(df_len):
                c = cosine_similarity(q, feature_tfidf[i])
                sim.append(c)

            sim_idx = np.argmax(sim)   
            sim_score = np.max(sim)

            result_score.append(sim_score)
            result_name.append(list(p_name)[sim_idx])
        names['주방'] = result_name
        scores['주방'] = result_score   
        
    elif group == '가전':
        p_name = np.unique(가전['group_p_name'])

        vectorizer = CountVectorizer(ngram_range=(1,5))
        feature_vector = vectorizer.fit_transform(p_name)
        print(feature_vector.shape)
        vocab = vectorizer.get_feature_names()

        transformer = TfidfTransformer(smooth_idf=False)
        feature_tfidf = transformer.fit_transform(feature_vector)

        test_name = 가전_test['group_p_name']
        df_len = len(p_name)

        result_score = []
        result_name = []
        
        for name in test_name:
            query = [name]
            q = vectorizer.transform(query)

            sim = []
            for i in range(df_len):
                c = cosine_similarity(q, feature_tfidf[i])
                sim.append(c)

            sim_idx = np.argmax(sim)   
            sim_score = np.max(sim)

            result_score.append(sim_score)
            result_name.append(list(p_name)[sim_idx])
        names['가전'] = result_name
        scores['가전'] = result_score 

    elif group == '침구':
        p_name = np.unique(침구['group_p_name'])

        vectorizer = CountVectorizer(ngram_range=(1,5))
        feature_vector = vectorizer.fit_transform(p_name)
        print(feature_vector.shape)
        vocab = vectorizer.get_feature_names()

        transformer = TfidfTransformer(smooth_idf=False)
        feature_tfidf = transformer.fit_transform(feature_vector)

        test_name = 침구_test['group_p_name']
        df_len = len(p_name)

        result_score = []
        result_name = []
        
        for name in test_name:
            query = [name]
            q = vectorizer.transform(query)

            sim = []
            for i in range(df_len):
                c = cosine_similarity(q, feature_tfidf[i])
                sim.append(c)

            sim_idx = np.argmax(sim)   
            sim_score = np.max(sim)

            result_score.append(sim_score)
            result_name.append(list(p_name)[sim_idx])
        names['침구'] = result_name
        scores['침구'] = result_score 

    elif group == '잡화':
        p_name = np.unique(잡화['group_p_name'])

        vectorizer = CountVectorizer(ngram_range=(1,5))
        feature_vector = vectorizer.fit_transform(p_name)
        print(feature_vector.shape)
        vocab = vectorizer.get_feature_names()

        transformer = TfidfTransformer(smooth_idf=False)
        feature_tfidf = transformer.fit_transform(feature_vector)

        test_name = 잡화_test['group_p_name']
        df_len = len(p_name)

        result_score = []
        result_name = []
        
        for name in test_name:
            query = [name]
            q = vectorizer.transform(query)

            sim = []
            for i in range(df_len):
                c = cosine_similarity(q, feature_tfidf[i])
                sim.append(c)

            sim_idx = np.argmax(sim)   
            sim_score = np.max(sim)

            result_score.append(sim_score)
            result_name.append(list(p_name)[sim_idx])
        names['잡화'] = result_name
        scores['잡화'] = result_score     

    else:
        continue

names['가전'] = [i.replace('가전 1등급 221l_딤채 김치 냉장고 900000', '가전 일시불 대우전자 벽걸이 에어컨 tdo z-s 10jk 810000') for i in names['가전']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


(352, 4123)
(219, 3215)
(144, 2462)
(56, 921)
(180, 2373)
(101, 946)
(60, 595)
(174, 2643)
(284, 3870)
(51, 814)
(230, 3113)



In [ ]:
for group in p_group_name :
    
    if group == '주방':
        주방_names = names['주방']
        temp = []
        for name in 주방_names:
            matching_code = np.unique(주방[주방['group_p_name'] == name]['brand_name_code'])[0]
            temp.append(matching_code)
        주방_test['brand_name_code'] = temp

    
    elif group == '건강기능':
        건강기능_names = names['건강기능']
        temp = []
        for name in 건강기능_names:
            matching_code = np.unique(건강기능[건강기능['group_p_name'] == name]['brand_name_code'])[0]
            temp.append(matching_code)
        건강기능_test['brand_name_code'] = temp
    
    if group == '잡화':
        잡화_names = names['잡화']
        temp = []
        for name in 잡화_names:
            matching_code = np.unique(잡화[잡화['group_p_name'] == name]['brand_name_code'])[0]
            temp.append(matching_code)
        잡화_test['brand_name_code'] = temp
    
    elif group == '생활용품':
        생활용품_names = names['생활용품']
        temp = []
        for name in 생활용품_names:
            matching_code = np.unique(생활용품[생활용품['group_p_name'] == name]['brand_name_code'])[0]
            temp.append(matching_code)
        생활용품_test['brand_name_code'] = temp
    
    elif group == '침구':
        침구_names = names['침구']
        temp = []
        for name in 침구_names:
            matching_code = np.unique(침구[침구['group_p_name'] == name]['brand_name_code'])[0]   
            temp.append(matching_code)
        침구_test['brand_name_code'] = temp
    
    elif group == '가구':
        가구_names = names['가구']
        temp = []
        for name in 가구_names:
            matching_code = np.unique(가구[가구['group_p_name'] == name]['brand_name_code'])[0]
            temp.append(matching_code)
        가구_test['brand_name_code'] = temp
    
    elif group == '이미용':
        이미용_names = names['이미용']
        temp = []
        for name in 이미용_names:
            matching_code = np.unique(이미용[이미용['group_p_name'] == name]['brand_name_code'])[0]  
            temp.append(matching_code)
        이미용_test['brand_name_code'] = temp
    
    elif group == '농수축':
        농수축_names = names['농수축']
        temp = []
        for name in 농수축_names:
            matching_code = np.unique(농수축[농수축['group_p_name'] == name]['brand_name_code'])[0]
            temp.append(matching_code)
        농수축_test['brand_name_code'] = temp
    
    elif group == '속옷':
        속옷_names = names['속옷']
        temp = []
        for name in 속옷_names:
            matching_code = np.unique(속옷[속옷['group_p_name'] == name]['brand_name_code'])[0]
            temp.append(matching_code)
        속옷_test['brand_name_code'] = temp   
        
    elif group == '의류':
        의류_names = names['의류']
        temp = []
        for name in 의류_names:
            matching_code = np.unique(의류[의류['group_p_name'] == name]['brand_name_code'])[0]
            temp.append(matching_code)
        의류_test['brand_name_code'] = temp
        
    elif group == '가전':
        가전_names = names['가전']
        temp = []
        for name in 가전_names:
            matching_code = np.unique(가전[가전['group_p_name'] == name]['brand_name_code'])[0]
            temp.append(matching_code)
        가전_test['brand_name_code'] = temp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [ ]:
for group in p_group_name :
    
    if group == '주방':
        주방_names = names['주방']
        temp = []
        for name in 주방_names:
            matching_code = np.unique(주방[주방['group_p_name'] == name]['brand_power'])[0]
            temp.append(matching_code)
        주방_test['brand_power'] = temp

    
    elif group == '건강기능':
        건강기능_names = names['건강기능']
        temp = []
        for name in 건강기능_names:
            matching_code = np.unique(건강기능[건강기능['group_p_name'] == name]['brand_power'])[0]
            temp.append(matching_code)
        건강기능_test['brand_power'] = temp
    
    if group == '잡화':
        잡화_names = names['잡화']
        temp = []
        for name in 잡화_names:
            matching_code = np.unique(잡화[잡화['group_p_name'] == name]['brand_power'])[0]
            temp.append(matching_code)
        잡화_test['brand_power'] = temp
    
    elif group == '생활용품':
        생활용품_names = names['생활용품']
        temp = []
        for name in 생활용품_names:
            matching_code = np.unique(생활용품[생활용품['group_p_name'] == name]['brand_power'])[0]
            temp.append(matching_code)
        생활용품_test['brand_power'] = temp
    
    elif group == '침구':
        침구_names = names['침구']
        temp = []
        for name in 침구_names:
            matching_code = np.unique(침구[침구['group_p_name'] == name]['brand_power'])[0]   
            temp.append(matching_code)
        침구_test['brand_power'] = temp
    
    elif group == '가구':
        가구_names = names['가구']
        temp = []
        for name in 가구_names:
            matching_code = np.unique(가구[가구['group_p_name'] == name]['brand_power'])[0]
            temp.append(matching_code)
        가구_test['brand_power'] = temp
    
    elif group == '이미용':
        이미용_names = names['이미용']
        temp = []
        for name in 이미용_names:
            matching_code = np.unique(이미용[이미용['group_p_name'] == name]['brand_power'])[0]  
            temp.append(matching_code)
        이미용_test['brand_power'] = temp
    
    elif group == '농수축':
        농수축_names = names['농수축']
        temp = []
        for name in 농수축_names:
            matching_code = np.unique(농수축[농수축['group_p_name'] == name]['brand_power'])[0]
            temp.append(matching_code)
        농수축_test['brand_power'] = temp
    
    elif group == '속옷':
        속옷_names = names['속옷']
        temp = []
        for name in 속옷_names:
            matching_code = np.unique(속옷[속옷['group_p_name'] == name]['brand_power'])[0]
            temp.append(matching_code)
        속옷_test['brand_power'] = temp   
        
    elif group == '의류':
        의류_names = names['의류']
        temp = []
        for name in 의류_names:
            matching_code = np.unique(의류[의류['group_p_name'] == name]['brand_power'])[0]
            temp.append(matching_code)
        의류_test['brand_power'] = temp
        
    elif group == '가전':
        가전_names = names['가전']
        temp = []
        for name in 가전_names:
            matching_code = np.unique(가전[가전['group_p_name'] == name]['brand_power'])[0]
            temp.append(matching_code)
        가전_test['brand_power'] = temp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [ ]:
무형_test = test[test['p_group'] == '무형']
무형_test['brand_name_code'] = ''
무형_test['brand_power'] = ''

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
result_all = pd.concat([의류_test, 속옷_test, 농수축_test, 이미용_test, 가전_test, 가구_test, 침구_test, 생활용품_test, 잡화_test, 건강기능_test, 주방_test, 무형_test])
del result_all['split_p_name']
del result_all['u']
del result_all['group_p_name']
#del data['Unnamed: 0']
data = result_all.sort_index()
data = data[['brand_name_code', 'brand_power']]

In [ ]:
df_e = pd.concat([df_e,data],axis=1)

In [ ]:
df_e

,m_code,p_code,unit_price,total_price,MONTH,DAY,HOUR,MINUTE,DAY_NUM,p_group_code,day_of_week_and_hour,52_week\t,holiday,timeslot,season,rain,mask_ratio,parcel_ratio,fill_exp_min,div_exp_min,brand_name_code,brand_power
0,100650,201971,59800,NaN,6,1,6,20,1,0,6,23,0,1,2,0,1.85848,1.15168,20.000000,20.000000,306,20.9232
1,100650,201971,59800,NaN,6,1,6,40,1,0,6,23,0,1,2,0,1.85848,1.15168,20.000000,20.000000,306,20.9232
2,100650,201971,59800,NaN,6,1,7,0,1,0,7,23,0,1,2,0,1.85848,1.15168,20.000000,20.000000,306,20.9232
3,100445,202278,69900,NaN,6,1,7,20,1,1,7,23,0,1,2,0,1.85848,1.15168,20.000000,20.000000,353,26.3041
4,100445,202278,69900,NaN,6,1,7,40,1,1,7,23,0,1,2,0,1.85848,1.15168,20.000000,20.000000,353,26.3041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,100660,201989,0,NaN,7,1,0,20,3,5,48,27,0,7,2,0,1.11449,1.81406,20.000000,20.000000,,
2887,100660,201989,0,NaN,7,1,0,40,3,5,48,27,0,7,2,0,1.11449,1.81406,20.000000,20.000000,,
2888,100660,201989,0,NaN,7,1,1,0,3,5,49,27,0,7,2,0,1.11449,1.81406,20.000000,20.000000,,
2889,100261,200875,69900,NaN,7,1,1,20,3,0,49,27,0,7,2,0,1.11449,1.81406,20.000000,20.000000,166,24.3873


In [ ]:
# 모델을 돌릴때 불필요한 feature 들을 제거해준다.
df_e.drop(['broadcast', 'exp_min', 'p_name', 'p_group', 'YEAR', 'SECOND', 'YEAR_DAY', 'DAY_NAME', 'code', 'no', 'quarter'], axis='columns', inplace=True)

In [ ]:
df_e.rename(columns = {'52_week\t' : '52_week'}, inplace = True)

In [ ]:
df_e = df_e[['m_code', 'p_code', 'unit_price', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'DAY_NUM', 'p_group_code', 'day_of_week_and_hour', 'rain', '52_week', 'holiday', 'brand_name_code', 'brand_power', 'mask_ratio', 'parcel_ratio', 'timeslot', 'season', 'fill_exp_min', 'div_exp_min', 'total_price']]

In [ ]:
# 생성한 데이터를 각각 파일로 저장한다.
df.to_excel('./bc/base_line_model.xlsx', index=False)
df_e.to_excel('./bc/base_line_eval_model.xlsx', index=False)